In [59]:
import tweepy
from pymongo import MongoClient
import json
import re
import string
import pandas as pd
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

consumer_key = "POA1eXAkYisz3kBRdcmjeBcha"
consumer_secret = "Ol97n9dfJcSiHvxUmPfRoR7QcLBMzuM5MdOrG36dBuNyT2EzNw"
access_token = "354671169-ROmqMvVamNyggX9hqNzlpqhTgc6AkCOpT6bsjodA"
access_secret = "RqlakpclhdjQzijYhaSfqqK0Yhzv3AZe6RdQggdms8sYn"
 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.secure = True
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

client = MongoClient('localhost', 27017)
db = client.bigData

In [103]:
class MyStreamListener(tweepy.StreamListener):

    print('func called')
        
    def on_data(self, data):
        # Decode JSON
        client = MongoClient('localhost', 27017)
        db = client.bigData
        
        datajson = json.loads(data)
        if 'lang' in datajson:
            if datajson['lang'] == 'en':
                #print datajson['text']
                data_insert = {'tweet_text': datajson['text']}
                db.tweetUSA.insert(data_insert)
                print("inserted")

     
        
myStreamListener = MyStreamListener()

myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

GEOBOX_WORLD = [-180,-90,180,90]
#format = [leftLimit   lowerLimit    rightLimit     upperLimit]
GEOBOX_INDIA = [71.1924, 8.0883,94.7278, 33.7782]
#68.11	6.46	97.4	35.51
#-9.4	18.8	113.1	71.5
#-167.1	12.0	-53.0	71.6
GEOBOX_USA = [-167.1, 12.8, -53 , 71.5]
GEOBOX_AUS  = [112.6,-41.2,156.7,-10.5]
myStream.filter(locations = GEOBOX_USA)

func called


/home/fsftn/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
i

IncompleteRead: IncompleteRead(0 bytes read, 1 more expected)

In [98]:
def clean_text(text):
    text = text.encode('ascii', 'ignore')
    text = ' '.join([word for word in text.split() if word not in cachedStopWords])
    

    
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    
    
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    text = ' '.join([word for word in text.split() if len(word) >=3 ])

    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


In [99]:
def find_tweets(location):
    cursor = []
    if location == 'AUS':
        cursor  = db.tweetAUS.find({})
    if location == 'IND':
        cursor  = db.tweetINDIA.find({})
    if location == 'USA':
        print 'in USA'
        cursor = db.tweetUSA.find({})
    print cursor
        
    tweet_fields = ['tweet_text']
    docs = pd.DataFrame(list(cursor), columns = tweet_fields)
    docs  = docs['tweet_text'].tolist()
    return docs

In [100]:
aus = find_tweets('USA')
aus = [clean_text(tweet) for tweet in aus]
print len(aus)

in USA
2973


In [101]:
import textmining
import lda
model = lda.LDA(n_topics=5 , n_iter=150 , random_state=1)
import numpy as np

def termdocumentmatrix_example(docs):
    # Create some very short sample documents
    tdm = textmining.TermDocumentMatrix()
    for doc in docs:
        tdm.add_doc(doc)
    
    '''
    doc1 = 'John and Bob are brothers.'
    doc2 = 'John went to the store. The store was closed.'
    doc3 = 'Bob went to the store too.'
    # Initialize class to create term-document matrix
    
    # Add the documents
    tdm.add_doc(doc1)
    tdm.add_doc(doc2)
    tdm.add_doc(doc3)
    '''
    dtrow= []
    cnt = 0 
    for row in tdm.rows(cutoff=2):
        if cnt == 0 :
            vocab = row
        else:
            dtrow.append(row)
        #print row
        cnt+=1
    dtrow = np.array(dtrow)
    return vocab,dtrow
vocab, dtrow =  termdocumentmatrix_example(aus)

In [102]:
model = lda.LDA(n_topics=5, n_iter= 1200 , random_state=1)

model.fit(dtrow)

topic_word = model.topic_word_
n_top_words = 8

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 2973
INFO:lda:vocab_size: 2229
INFO:lda:n_words: 14853
INFO:lda:n_topics: 5
INFO:lda:n_iter: 1200
INFO:lda:<0> log likelihood: -162566
INFO:lda:<10> log likelihood: -120121
INFO:lda:<20> log likelihood: -118239
INFO:lda:<30> log likelihood: -117223
INFO:lda:<40> log likelihood: -116782
INFO:lda:<50> log likelihood: -116553
INFO:lda:<60> log likelihood: -116195
INFO:lda:<70> log likelihood: -116200
INFO:lda:<80> log likelihood: -115939
INFO:lda:<90> log likelihood: -115640
INFO:lda:<100> log likelihood: -115654
INFO:lda:<110> log likelihood: -115537
INFO:lda:<120> log likelihood: -115437
INFO:lda:<130> log likelihood: -115126
INFO:lda:<140> log likelihood: -115106
INFO:lda:<150> log likelihood: -115299
INFO:lda:<160> log likelihood: -115200
INFO:lda:<170> log likelihood: -114991
INFO:lda:<180> log likelihood: -114760
INFO:lda:<190> log likelihood: -114784
INFO:lda:<200> log likelihood: -114839
INFO:lda:<210> log likelihood: -114711
INFO:lda:<220> log likelihood: -1

Topic 0: want work details got click today the f
Topic 1: get like good people the don back know
Topic 2: latest opening here see today amp read new
Topic 3: you great can this anyone fit happy recommend
Topic 4: like get lol man don amp never take
